<a href="https://colab.research.google.com/github/fitahw/ML/blob/main/titanic_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers

from matplotlib import pyplot as plt

tf.keras.backend.set_floatx('float32')

train_df=pd.read_csv(filepath_or_buffer='/content/train.csv')
# Set the ratio for splitting (e.g., 80% training, 20% validation)
split_ratio = 0.8

# Determine the number of samples for training
num_train = int(len(train_df) * split_ratio)

train_df = train_df.sample(frac=1).reset_index(drop=True)
# Split the data
training_df = train_df.iloc[:num_train]
val_df = train_df.iloc[num_train:]

test_df = pd.read_csv(filepath_or_buffer='/content/test.csv')

training_df.head(n=5)
y = training_df['Survived'].values
training_df.drop(columns='Survived', inplace=True)

<ipython-input-6-81c1c8a1240c>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_df.drop(columns='Survived', inplace=True)


In [ ]:
def cleanup_df(training_df):
  training_df['Cabin'].fillna('N', inplace=True)
  training_df['Deck'] = training_df['Cabin'].apply(lambda x:str(x)[:1])
  training_df['Room'] = training_df['Cabin'].apply(lambda x:str(x).replace(str(x)[0], ''))

  training_df = pd.get_dummies(training_df, columns=['Deck', 'Embarked'])
  training_df = pd.get_dummies(training_df, columns=['Sex'], drop_first=True)

  training_df.drop(columns='Cabin', inplace=True)
  training_df.drop(columns='Ticket', inplace=True)
  training_df.drop(columns='PassengerId', inplace=True)
  training_df.drop(columns='Name', inplace=True)

  training_df['Age'].fillna(training_df['Age'].mean(), inplace=True)
  training_df['Age'] = training_df['Age'].astype('int64')

  training_df['Room'] = training_df['Room'].apply(lambda x:str(x).split(' ', 1)[0])
  training_df['Room'] = training_df['Room'].replace('', '0')
  training_df['Room'] = training_df['Room'].astype('int64')

  return training_df

In [ ]:
training_df = cleanup_df(training_df)
test_df = cleanup_df(test_df)
training_df.drop(columns='Deck_T', inplace=True)

In [ ]:
training_df.dtypes

Pclass          int64
Age             int64
SibSp           int64
Parch           int64
Fare          float64
Room            int64
Deck_A          uint8
Deck_B          uint8
Deck_C          uint8
Deck_D          uint8
Deck_E          uint8
Deck_F          uint8
Deck_G          uint8
Deck_N          uint8
Embarked_C      uint8
Embarked_Q      uint8
Embarked_S      uint8
Sex_male        uint8
dtype: object

In [ ]:
learning_rate = 0.01
epochs = 200
batch_size = 16
'''
SibSp_Parch_cross = tf.keras.layers.HashedCrossing(
    num_bins=training_df['SibSp'].max() * training_df['Parch'].max(),
    output_mode='one_hot',
    name='cross_SibSp_Parch')([inputs.get('SibSp'), inputs.get('Parch')])\
'''

model = tf.keras.Sequential([
    layers.Input(shape=(training_df.shape[1],)),
    layers.Dense(18, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.experimental.RMSprop(
    learning_rate=learning_rate),
    loss="mean_squared_error",
    metrics=[tf.keras.metrics.RootMeanSquaredError()])

history = model.fit(training_df, y, epochs=epochs, batch_size=batch_size)

Epoch 1/200
56/56 [==============================] - 1s 2ms/step - loss: 1.1318 - accuracy: 0.5960
Epoch 2/200
56/56 [==============================] - 0s 2ms/step - loss: 0.7199 - accuracy: 0.6667
Epoch 3/200
56/56 [==============================] - 0s 2ms/step - loss: 0.7638 - accuracy: 0.6801
Epoch 4/200
56/56 [==============================] - 0s 2ms/step - loss: 0.6989 - accuracy: 0.6723
Epoch 5/200
56/56 [==============================] - 0s 2ms/step - loss: 0.6490 - accuracy: 0.6633
Epoch 6/200
56/56 [==============================] - 0s 2ms/step - loss: 0.6309 - accuracy: 0.6790
Epoch 7/200
56/56 [==============================] - 0s 2ms/step - loss: 0.6374 - accuracy: 0.6689
Epoch 8/200
56/56 [==============================] - 0s 3ms/step - loss: 0.5759 - accuracy: 0.6925
Epoch 9/200
56/56 [==============================] - 0s 2ms/step - loss: 0.5792 - accuracy: 0.6813
Epoch 10/200
56/56 [==============================] - 0s 2ms/step - loss: 0.5675 - accuracy: 0.6902
Epoch 11/

In [ ]:
predictions = model.predict(test_df)
threshold = 0.65
boolean_predictions = (predictions >= threshold).astype(int)
predictions_df = pd.DataFrame(boolean_predictions, columns=['Survived'])

predictions_df.head(n=10)

14/14 [==============================] - 0s 2ms/step


,Survived
0,0
1,0
2,0
3,0
4,0
5,0
6,1
7,0
8,1
9,0


In [ ]:
test_df = pd.read_csv(filepath_or_buffer='/content/test.csv')

result_df = pd.concat([predictions_df, test_df['PassengerId']], axis=1)
result_df.head(n=10)

,Survived,PassengerId
0,0,892
1,0,893
2,0,894
3,0,895
4,0,896
5,0,897
6,1,898
7,0,899
8,1,900
9,0,901


In [ ]:
result_df.to_csv('Result.csv', index=False)